In [22]:
import mysql.connector
from mysql.connector import errorcode
import psycopg2
from flask import Flask, request
from flask_limiter import Limiter, util
from hashlib import sha256
import random
import string
import time
import json
import datetime 

In [23]:
cnx = psycopg2.connect(database="hehe", 
                            user="koyeb-adm", 
                            password="uq2lZvhI6FPR", 
                            host="ep-soft-waterfall-48511697.eu-central-1.pg.koyeb.app")
cursor = cnx.cursor()
cursor.execute("ROLLBACK")
cnx.commit()

In [24]:
cursor.execute('''CREATE TABLE IF NOT EXISTS users ( \
          userId SERIAL PRIMARY KEY, \
          username VARCHAR(256) UNIQUE NOT NULL, \
          password VARCHAR(256) NOT NULL, \
          loggedIn SMALLINT DEFAULT 0, \
          loggedAt TIMESTAMP DEFAULT NULL); ''')
        


In [25]:
usernames = set([])
users = []
add_user = ("INSERT INTO users "
                "(username, password) "
                "VALUES ( %s, %s)")
cursor.execute(f"SELECT COUNT(*) FROM users")
number = cursor.fetchone()
# print(number)
if number == (1000000,):
    print('Already init') 
else:
    while(len(usernames) < 10000):
        username = ''.join((random.choice(string.ascii_lowercase) for x in range(6)))
        usernames.add(username)
        

    for username in usernames:
        password = sha256(''.join(str(random.randint(0, 9)) for i in range(6)).encode('utf-8')).hexdigest() 
        users.append((username, password))

    # Disable autocommit
    # cnx.autocommit = False
    l = len(users)
    batch_size = l//10
    start_time = time.time()
    for i in range(0, l, batch_size):
        batch = users[i:i + batch_size]
        cursor.executemany(add_user, batch)
        cnx.commit()
    end_time = time.time()

    res = f"Total insertion time: {end_time - start_time} seconds"

    # Re-enable autocommit
    # cnx.autocommit = True

KeyboardInterrupt: 